In [1]:
import pymc3 as pm 
import matplotlib.pyplot as plt
import numpy as np 
import theano
import theano.tensor as tt
from scipy import stats
import pymc3 as pm 
from ipynb.fs.full.simul_data import *
from ipynb.fs.full.counterfactual_generate import *
a_dim=7
mu_a = np.zeros(a_dim)
cov_a = np.eye(a_dim)

num_samples =1000
trans_dim = 60
u_dim = 30
rating_dim = 50
N=100
mu_u = np.zeros(u_dim)
cov_u = np.eye(u_dim)
mu_trans = np.zeros(trans_dim)
cov_trans = np.eye(trans_dim)
mu_rating = np.zeros(rating_dim)
cov_rating = np.eye(rating_dim)

In [2]:

causal_model = model1(u_dim,trans_dim,rating_dim)
data = causal_model.generate(num_samples)

print("transcript shape ",data["transcript"].shape)
print("view shape ",data["view"].shape)
print("a shape ",data["a"].shape)
print("rating shape ",data["rating"].shape)
# mu_rand = np.random.normal(size=10)

transcript shape  (1000, 60)
view shape  (1000,)
a shape  (1000, 7)
rating shape  (1000, 50)


In [3]:
def model_fit(data,method='mcmc'):

    with pm.Model() as model:

        #prior sampling
        u = pm.MvNormal('u',mu=mu_u,cov=cov_u,shape=u_dim)
        transcript0 = pm.MvNormal('transcript0',mu=mu_trans,cov=cov_trans,shape=trans_dim)
        view0 = pm.Normal('view0',mu=0,tau=1)

        #effect of u 
        eta_u_transcript = pm.MatrixNormal('eta_u_transcript',colcov=cov_u, rowcov=cov_trans, shape=(trans_dim, u_dim))
        eta_u_view = pm.MvNormal('eta_u_view',mu=mu_u,cov=cov_u,shape=u_dim)
        eta_u_rating = pm.MatrixNormal('eta_u_rating',colcov=cov_u, rowcov=cov_rating, shape=(rating_dim, u_dim))

        #effect of protected attribute
        eta_a_transcript = pm.MatrixNormal('eta_a_transcript',colcov=cov_trans, rowcov=cov_a, shape=(a_dim, trans_dim))
        eta_a_view = pm.MvNormal('eta_a_view',mu=mu_a,cov=cov_a,shape=a_dim)
        eta_a_rating = pm.MatrixNormal('eta_a_rating',colcov=cov_rating, rowcov=cov_a, shape=(a_dim,rating_dim))

        #effect of transcript on view and rating
        eta_transcript_view = pm.MvNormal('eta_transcript_view',mu=mu_trans,cov=cov_trans,shape=trans_dim)
        eta_transcript_rating = pm.MatrixNormal('eta_transcript_rating',colcov=cov_rating, rowcov=cov_trans, shape=(trans_dim,rating_dim))

        #effect of view on rating
        eta_view_rating = pm.MvNormal('eta_view_rating',mu=mu_rating,cov=cov_rating,shape=rating_dim)


        sigma_transcript_sq = pm.InverseGamma('sigma_transcript_sq',alpha=1,beta=1)
        sigma_rating_sq = pm.InverseGamma('sigma_rating_sq',alpha=1,beta=1)
        #print(data['a'].shape)
        
        transcript_mean =   tt.dot(eta_u_transcript ,u)+transcript0 + tt.dot(data['a'] , eta_a_transcript)
        transcript = pm.MvNormal('transcript', mu= transcript_mean, cov = sigma_transcript_sq*np.eye(trans_dim), observed = data["transcript"] )
        
        
        view_mean = tt.maximum(1,view0 + tt.dot(eta_u_view , u)+   tt.dot(data['a'],eta_a_view)+tt.dot(transcript, eta_transcript_view)) 
        view = pm.Poisson('view',mu =view_mean, observed = data['view'] )

        rating_mean = tt.dot(eta_u_rating , u) +  tt.dot(data['a'],eta_a_rating) + tt.dot(transcript, eta_transcript_rating) + tt.dot(tt.reshape(view,(-1,1)), tt.reshape(eta_view_rating,(1,-1))) 
        rating = pm.MvNormal('rating', mu= rating_mean, cov = sigma_rating_sq*np.eye(rating_dim), observed = data["rating"] ) 
        
        if method == 'mcmc':   
            step = pm.NUTS()
            #init='adapt_diag'
            trace=pm.sample(100)#,step =step)#,target_accept=0.8)
        elif method == 'vi':
            trace=pm.fit(n=300)
        else:
            print('Not Implemented any other method')
            return
            
    return trace
    

In [4]:
mf = model_fit(data,'vi')

Average Loss = 3.0414e+08: 100%|██████████| 300/300 [00:02<00:00, 122.63it/s]
Finished [100%]: Average Loss = 3.0384e+08


In [5]:
trace = mf.sample(1000)

In [6]:
for key in causal_model.params_dic:
    try:
        s=causal_model.params_dic[key].shape
        if len(s)==1:
            s=s[0]
        else:
            s=s[0]*s[1]
    except:
        s=1
#     if len(s)==2:
#         s=s[0]*s[2]
#     else:
#         s=s[0]
    diff = np.linalg.norm(causal_model.params_dic[key]-np.mean(trace[key],axis=0)) #/ s
    print("Difference for ",key," is ", diff)
    

Difference for  transcript0  is  7.789389761579012
Difference for  view0  is  0.33407142451478977
Difference for  eta_u_transcript  is  42.69178997224917
Difference for  eta_u_view  is  4.831350743493851
Difference for  eta_u_rating  is  39.97560383146462
Difference for  eta_a_transcript  is  19.592831856098123
Difference for  eta_a_view  is  4.252498750767194
Difference for  eta_a_rating  is  18.986756078870712
Difference for  eta_transcript_view  is  7.724349562035345
Difference for  eta_transcript_rating  is  53.9837905949252
Difference for  eta_view_rating  is  5.431330219994437
Difference for  sigma_transcript_sq  is  0.6908104035868758
Difference for  sigma_rating_sq  is  0.27116003670147426


In [7]:
causal_model.params_dic.keys()

dict_keys(['transcript0', 'view0', 'eta_u_transcript', 'eta_u_view', 'eta_u_rating', 'eta_a_transcript', 'eta_a_view', 'eta_a_rating', 'eta_transcript_view', 'eta_transcript_rating', 'eta_view_rating', 'sigma_transcript_sq', 'sigma_rating_sq'])

In [9]:
counterfactual_sample(data,trace)

num_original_sample is  1000


AttributeError: 'numpy.ndarray' object has no attribute 'broadcastable'